In [ ]:
# # intallation
# %pip install --quiet google-ai-generativelanguage==0.6.15
# %pip install --upgrade --quiet google-generativeai
# %pip install --upgrade --quiet langchain_community
# %pip install --upgrade --quiet PyPDF2
# %pip install --upgrade --quiet langchain-chroma
# %pip install --upgrade --quiet pypdf

In [ ]:
# %pip install --upgrade --quiet langchain_community

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# imports
import getpass
import os

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains.llm import LLMChain
# LangChain I/O
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
# LCEL
from langchain.schema.runnable import RunnablePassthrough
# Summerize document
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
import textwrap

In [2]:
# Load documents
file_path = './Corporate Governance in India.pdf'
loader = PyPDFLoader(file_path)
documents = loader.load()
print(f"Loaded {len(documents)} documents from the folder.")

Loaded 64 documents from the folder.


In [3]:
# setup Google Generative AI
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API key here ")

In [4]:
# create LLM instance
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [5]:
# test
test_llm_response = llm.invoke("Tell me a joke about economics")
print(test_llm_response.content)

Here are a few economics jokes for you:

1.  **What's the difference between an economist and a dog?**
    The dog only has one hand.

2.  An economist, a physicist, and a chemist are stranded on a desert island with a can of food but no opener.
    *   The physicist says, "We can drop it from a high tree and use the force of impact."
    *   The chemist says, "We can build a fire and heat it to expand the metal until it bursts."
    *   The economist says, "Assume we have a can opener..."

3.  **What's the definition of an economist?**
    Someone who explains tomorrow why the predictions they made yesterday didn't come true today.

4.  **What's an economist's favorite pick-up line?**
    "I've got the supply, baby, if you've got the demand."


In [6]:
# create embeddings model
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [9]:
# Create Chroma vector store from documents
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="my_document_collection",
    persist_directory="./chroma_db"
)

In [12]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever.invoke("What this corporate gevernance?")

[Document(metadata={'source': './Corporate Governance in India.pdf', 'author': 'mk sridhar', 'total_pages': 64, 'creator': 'Microsoft® Word 2016', 'page': 15, 'moddate': '2022-01-28T06:21:12+00:00', 'page_label': '16', 'producer': 'www.ilovepdf.com', 'creationdate': '2022-01-28T06:21:12+00:00'}, page_content='12 \n \n \n \n \n1. The Principles of Governance \n \n \nThe OECD Principles of Corporate Governance, endorsed by OECD  Ministries have since become an \ninternational benchmark for policy makers, investors, corporations and other stakeholders \nworldwide. They have advanced the corporate governance agenda and provided specific guidance for \nlegislative and regulatory initiatives in both OECD and non-OECD countries. \n \nThese principles are broadly: \n \n \nSource:http://www.oecd.org/daf/ca/Corproate-Governance-Principles-ENG.pdf'),
 Document(metadata={'producer': 'www.ilovepdf.com', 'source': './Corporate Governance in India.pdf', 'page_label': '21', 'total_pages': 64, 'moddate

In [13]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(template)

In [14]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt
)
response = rag_chain.invoke("What this corporate gevernance?")

In [15]:
response

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n[Document(metadata={'total_pages': 64, 'source': './Corporate Governance in India.pdf', 'page_label': '16', 'creator': 'Microsoft® Word 2016', 'creationdate': '2022-01-28T06:21:12+00:00', 'moddate': '2022-01-28T06:21:12+00:00', 'producer': 'www.ilovepdf.com', 'page': 15, 'author': 'mk sridhar'}, page_content='12 \\n \\n \\n \\n \\n1. The Principles of Governance \\n \\n \\nThe OECD Principles of Corporate Governance, endorsed by OECD  Ministries have since become an \\ninternational benchmark for policy makers, investors, corporations and other stakeholders \\nworldwide. They have advanced the corporate governance agenda and provided specific guidance for \\nlegislative and regulatory initiatives in both OECD and non-OECD countries. \\n \\nThese principles are broadly: \\n \\n \\nSource:http://www.oecd.org/daf/ca/Corproate-Governance-Principles-ENG.pdf'), Document(metadata={'page':

In [18]:
response.messages[0].content

"Answer the question based only on the following context:\n[Document(metadata={'total_pages': 64, 'source': './Corporate Governance in India.pdf', 'page_label': '16', 'creator': 'Microsoft® Word 2016', 'creationdate': '2022-01-28T06:21:12+00:00', 'moddate': '2022-01-28T06:21:12+00:00', 'producer': 'www.ilovepdf.com', 'page': 15, 'author': 'mk sridhar'}, page_content='12 \\n \\n \\n \\n \\n1. The Principles of Governance \\n \\n \\nThe OECD Principles of Corporate Governance, endorsed by OECD  Ministries have since become an \\ninternational benchmark for policy makers, investors, corporations and other stakeholders \\nworldwide. They have advanced the corporate governance agenda and provided specific guidance for \\nlegislative and regulatory initiatives in both OECD and non-OECD countries. \\n \\nThese principles are broadly: \\n \\n \\nSource:http://www.oecd.org/daf/ca/Corproate-Governance-Principles-ENG.pdf'), Document(metadata={'page': 20, 'source': './Corporate Governance in India

In [26]:
page_content="""page_content='17 \\n \\n \\nThe core concepts Corporate Governance can be summarised as: 
\\n \\na. Transparency \\n \\nTransparency leads to sufficient reporting without undermining the interest of the 
\\nclient. This needs the clarification of strategies and actions of the company for those \\nto whom it is 
accountable. The Financials are prepared in accordance with IFRS, \\nregistry filings are up-to-date, high 
quality Annual Reports are published,  Web based \\ndisclosure in place and timely accurate disclosures of 
all material matters are made. \\n \\nb. Rule of law \\n \\nUnderstand and abide by the various Rule of Law 
enacted. Anticipate future \\nrequirements and prepare well in time to ensure adherence. \\n \\nc. Participation 
\\n \\nEducate and involve all stakeholders in a fair manner while conducting the business. \\n \\nd. 
Responsiveness \\n \\nRespond in a timely, transparent and fair manner \\n \\ne. Responsibility. \\n \\nManagement 
Board is liable to shareholders and shareholders are liable to the \\nManagement Board. Responsibility gives 
success impetus. \\n \\nf. Equity \\n \\nBeing fair and practice equity. \\n \\ng. Ethics')"""


print(textwrap.fill(page_content,
            width=100,
            break_long_words=False,
            replace_whitespace=False))


page_content='17 \n \n \nThe core concepts Corporate Governance can be summarised as: 
\n \na.
Transparency \n \nTransparency leads to sufficient reporting without undermining the interest of the
\nclient. This needs the clarification of strategies and actions of the company for those \nto whom
it is 
accountable. The Financials are prepared in accordance with IFRS, \nregistry filings are up-
to-date, high 
quality Annual Reports are published,  Web based \ndisclosure in place and timely
accurate disclosures of 
all material matters are made. \n \nb. Rule of law \n \nUnderstand and
abide by the various Rule of Law 
enacted. Anticipate future \nrequirements and prepare well in time
to ensure adherence. \n \nc. Participation 
\n \nEducate and involve all stakeholders in a fair
manner while conducting the business. \n \nd. 
Responsiveness \n \nRespond in a timely, transparent
and fair manner \n \ne. Responsibility. \n \nManagement 
Board is liable to shareholders and
shareholders are liab

In [27]:
print(textwrap.fill(response.messages[0].content,
            width=100,
            break_long_words=False,
            replace_whitespace=False))

Answer the question based only on the following context:
[Document(metadata={'total_pages': 64,
'source': './Corporate Governance in India.pdf', 'page_label': '16', 'creator': 'Microsoft® Word
2016', 'creationdate': '2022-01-28T06:21:12+00:00', 'moddate': '2022-01-28T06:21:12+00:00',
'producer': 'www.ilovepdf.com', 'page': 15, 'author': 'mk sridhar'}, page_content='12 \n \n \n \n
\n1. The Principles of Governance \n \n \nThe OECD Principles of Corporate Governance, endorsed by
OECD  Ministries have since become an \ninternational benchmark for policy makers, investors,
corporations and other stakeholders \nworldwide. They have advanced the corporate governance agenda
and provided specific guidance for \nlegislative and regulatory initiatives in both OECD and non-
OECD countries. \n \nThese principles are broadly: \n \n
\nSource:http://www.oecd.org/daf/ca/Corproate-Governance-Principles-ENG.pdf'),
Document(metadata={'page': 20, 'source': './Corporate Governance in India.pdf', 'producer'

In [23]:
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [24]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
question = "What this corporate gevernance?"
response2 = rag_chain.invoke(question)
print(response2)

Based on the provided context, Corporate Governance can be understood in two ways:

1.  **The OECD Principles of Corporate Governance** are an international benchmark for policy makers, investors, corporations, and other stakeholders worldwide. They have advanced the corporate governance agenda and provided specific guidance for legislative and regulatory initiatives.

2.  **The core concepts of Corporate Governance** can be summarised as:
    *   **Transparency:** Sufficient reporting without undermining client interest, clarification of strategies and actions, financial reporting in accordance with IFRS, up-to-date registry filings, high-quality Annual Reports, web-based disclosure, and timely accurate disclosures of all material matters.
    *   **Rule of law:** Understanding and abiding by enacted laws, anticipating future requirements, and preparing for adherence.
    *   **Participation:** Educating and involving all stakeholders fairly.
    *   **Responsiveness:** Responding in 

In [28]:
print(textwrap.fill(response2,
            width=100,
            break_long_words=False,
            replace_whitespace=False))

Based on the provided context, Corporate Governance can be understood in two ways:

1.  **The OECD
Principles of Corporate Governance** are an international benchmark for policy makers, investors,
corporations, and other stakeholders worldwide. They have advanced the corporate governance agenda
and provided specific guidance for legislative and regulatory initiatives.

2.  **The core concepts
of Corporate Governance** can be summarised as:
    *   **Transparency:** Sufficient reporting
without undermining client interest, clarification of strategies and actions, financial reporting in
accordance with IFRS, up-to-date registry filings, high-quality Annual Reports, web-based
disclosure, and timely accurate disclosures of all material matters.
    *   **Rule of law:**
Understanding and abiding by enacted laws, anticipating future requirements, and preparing for
adherence.
    *   **Participation:** Educating and involving all stakeholders fairly.
    *
**Responsiveness:** Responding in a 

In [30]:
question2 = "How to achieve corporate gevernance in corporations?"
response3 = rag_chain.invoke(question2)
# print(response3)
print(textwrap.fill(response3,
            width=100,
            break_long_words=False,
            replace_whitespace=False))

Based on the provided context, corporate governance can be achieved by adhering to its core
concepts:

*   **Transparency:** By ensuring sufficient reporting without undermining client
interests, clarifying company strategies and actions, preparing financials in accordance with IFRS,
keeping registry filings up-to-date, publishing high-quality Annual Reports, having web-based
disclosure in place, and making timely accurate disclosures of all material matters.
*   **Rule of
law:** By understanding and abiding by various enacted laws, anticipating future requirements, and
preparing well in time to ensure adherence.
*   **Participation:** By educating and involving all
stakeholders in a fair manner while conducting the business.
*   **Responsiveness:** By responding
in a timely, transparent, and fair manner.
*   **Responsibility:** By ensuring the Management Board
is liable to shareholders and shareholders are liable to the Management Board.
*   **Equity:** By
being fair and practicing eq

In [33]:
# Summerize document
prompt_template2 = """Write a concise summary and with important concepts of the following:
"{text}"
"""
prompt2 = PromptTemplate.from_template(prompt_template2)

In [34]:
llm_chain2 = LLMChain(llm=llm, prompt=prompt2)

/tmp/ipykernel_12848/971568334.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain2 = LLMChain(llm=llm, prompt=prompt2)


In [35]:
stuff_chain2 = StuffDocumentsChain(
    llm_chain=llm_chain2,
    document_variable_name="text"
  )

/tmp/ipykernel_12848/4223527360.py:1: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  stuff_chain2 = StuffDocumentsChain(


In [36]:
output_summary2 = stuff_chain2.run(documents)

/tmp/ipykernel_12848/664794086.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output_summary2 = stuff_chain2.run(documents)


In [37]:
wrapped_text2 = textwrap.fill(output_summary2,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text2)

This e-book by CA. M K Sridhar, published by the Southern India Regional Council of The Institute of
Chartered Accountants of India, provides a comprehensive overview of Corporate Governance (CG) in
India.

**Concise Summary:**

The e-book "Corporate Governance in India" defines Corporate
Governance as a framework of internal controls, policies, and procedures designed to ensure
transparency, integrity, ethics, and honesty in a company's operations and dealings with
stakeholders. It details the evolution of CG in India, largely driven by the Ministry of Corporate
Affairs (MCA) and SEBI, through various committees (e.g., Kumaramangalam Birla, Naresh Chandra, N.R.
Narayana Murthy) and legislative acts like the Companies Act. Key concepts include the OECD
Principles of Corporate Governance, the "4 Pillars" (Vision & Mission, Stakeholder Engagement,
Compliance & Management, Professional Development & Succession), and the "4 P's" (People, Purpose,
Process, Performance). The book emphasizes 